In [22]:
# preinstaller
# mac - brew install sqlite3
# win - https://www.sqlite.org/download.html
import sys
from IPython.core.display import display, HTML

pkgs = ['bs4','selenium','chromedriver-binary==77.0.3865.40.0','ipywidgets',
        'numpy','pandas','itertools', 'tqdm'
        'ipython-autotime','ipython-sql'
       ]
for package in pkgs:
    try:
        import package
    except ImportError as error:
        # Install a pip package in the current Jupyter kernel
        !{sys.executable} -m pip install {package}
    except Exception as exception:
        # Output unexpected Exceptions.
        print(exception)
              
# why we choose sqllite
# https://blog.thedataincubator.com/2018/05/sqlite-vs-pandas-performance-benchmarks/


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that dat

You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. 

In [23]:
# imports
import json
import sqlite3
import os
import requests
from time import time
from multiprocessing.pool import ThreadPool
import tempfile
import subprocess
from os.path import relpath
import numpy as np 
import pandas as pd
import base64
import re 
import urllib
from requests.packages.urllib3.exceptions import InsecureRequestWarning


In [24]:
"""
debug a string with native default handler
# example open webbrowser
debug_str("<html><body><h3>hello world</h3></body></html>","html")
"""
def debug_str(mystr,filetype):
    if filetype is None:
        filetype="txt"
    tmp = tempfile.NamedTemporaryFile(delete=False,suffix="."+filetype)
    print(tmp)
    with open(tmp.name, 'w') as f:
        f.write(mystr)
    subprocess.call(["open", tmp.name])
    return tmp.name
"""
merge pandas by fuzzy match - taken from somewhere online
i would rather switch to a db for faster joinign
"""
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    return df_1

In [25]:
%%capture
# use tqdm integration for progress bars and widgets
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
# use autotime
%load_ext autotime

ImportError: No module named tqdm

In [26]:
# load yelp data into pandas
# ~1min 19s
# df_business = pd.read_csv('yelp_data/business.csv')
# df_review = pd.read_csv('yelp_data/review.csv')

# sqllite loading of data for sql
DB_FILE='./sql-lite-cache/foodcasting.db'
if os.path.exists(DB_FILE):
    print("exists!")
    # for the rest of the project we switch to sqlite3
    local_connection = sqlite3.connect(DB_FILE)
    c = local_connection.cursor()
else:
    # first time setup, we need to setup db
    os.makedirs('./sql-lite-cache', exist_ok=True)
    #   > git clone foodcasting
    #   > cd ~/git/foodcasting
    #   > $ sqlite3 sql-lite-cache/foodcasting.db
    #  sqlite> .read create-db-yelp.sql
 
    # setup db
    # load db
#     print("---setup---")
#     os.makedirs('./sql-lite-cache', exist_ok=True)
#     local_connection = sqlite3.connect(DB_FILE)
#     #local_connection.executescript(setup_sql)
#     setup_sql = open('create-db-yelp.sql', 'r').read()
#     print(setup_sql)
#     c = local_connection.cursor()
#     c.execute(setup_sql)
#     local_connection.commit()
#     c.close()
#     c = local_connection.cursor()
   



exists!


In [27]:
# use sql magics
%reload_ext sql
%sql sqlite:///sql-lite-cache/foodcasting.db
# now we can use magics to exec sql 
df_summary = %sql select * from summary
df_summary.DataFrame().head()


 * sqlite:///sql-lite-cache/foodcasting.db
Done.


,slug,categories,distance,name,price_level,rating,review_count,url,lat,lng,Sp1,type,homeurl,resource_id1,resource_id2,lat2,lng2
0,slug,categories,distance,name,price_level,rating,review_count,url,lat,lng,Sp1,type,homeurl,resource_id1,resource_id2,lat2,lng2
1,zizi-new-york-3,"Mediterranean, Middle Eastern",0.1 mi,Zizi,,4.875,16,https://m.yelp.com/biz/zizi-new-york-3,40.7429,-74,,natural,/biz/zizi-new-york-3,tM8ZtEix8Bxg82AIKsHXcw,yfrevmxjZ3Q-SahoFPJGoQ,40.7429,-74
2,chama-mama-new-york,Georgian,0.4 mi,Chama Mama,$$,4.39344,183,https://m.yelp.com/biz/chama-mama-new-york,40.7385,-73.9989,,natural,/biz/chama-mama-new-york,cTE0e96GYzet799EJeStoQ,-XdcVBshqkoiqgQWE1U3rQ,40.7385,-73.9989
3,hao-noodle-chelsea-new-york-7,"Chinese, Barbeque, Noodles",0.2 mi,Hao Noodle Chelsea,$$,4.32544,338,https://m.yelp.com/biz/hao-noodle-chelsea-new-...,40.7407,-74.0043,,natural,/biz/hao-noodle-chelsea-new-york-7,pCfGgX0WxGASGM4-Yhr2Pw,1JiTJvIgLvtE-Snfh9e8qA,40.7407,-74.0043
4,pepe-giallo-new-york,Italian,0.3 mi,Pepe Giallo,$$,3.9586,628,https://m.yelp.com/biz/pepe-giallo-new-york,40.7469,-74.005,,natural,/biz/pepe-giallo-new-york,AUifp6IDKzmorCM6vY2fug,0YD5gUeTbhheAbdc6hSUbA,40.7469,-74.005


In [28]:
df_menu = %sql select * from menu
df_menu.DataFrame().head()

 * sqlite:///sql-lite-cache/foodcasting.db
Done.


,slug,title,description,price
0,boqueria-fulton-market-chicago,Kas Naranja,,4
1,boqueria-fulton-market-chicago,Kas Manzana,,4
2,boqueria-fulton-market-chicago,Kas Limon,,4
3,boqueria-fulton-market-chicago,Vichy Catalan,1 liter.,8
4,boqueria-fulton-market-chicago,Mondariz,750 ml.,7


In [29]:
df_flat = %sql select t1.slug, t1.title || ' ' || t1.description || ' ' || t1.price AS menu_item from menu t1
df_flat.DataFrame().head(100)
df_flat.DataFrame().to_csv('./scrapy_yelp/concat_menu.csv', header=True, index=False, encoding='utf-8')

 * sqlite:///sql-lite-cache/foodcasting.db
Done.


In [30]:
df_s = %sql select t1.slug, group_concat(t1.menu_item, ', ') as menu from concat_menu t1 group by t1.slug
df_s.DataFrame().head()
df_s.DataFrame().to_csv('./scrapy_yelp/flat_menu.csv', header=True, index=False, encoding='utf-8')

 * sqlite:///sql-lite-cache/foodcasting.db
Done.


In [31]:
df = %sql select * from flat_menu t1 left join summary t2 on (t1.slug=t2.slug) where t2.rating IS NOT NULL
df.DataFrame().head()
df.DataFrame().to_csv('./scrapy_yelp/slug_menu.csv', header=True, index=False, encoding='utf-8')

 * sqlite:///sql-lite-cache/foodcasting.db
Done.


In [ ]:
import seaborn as sns
def visual_category1():
    cat = ['thai','chinese','japanese','korean','indian','american','caribbean','italian','mediterranean',
      'mexican','vietnamese','cajun','greek']
    df_name = ['thai_df','chinese_df','japanese_df','korean_df','indian_df','american_df','caribbean_df',
           'italian_df','mediterranean_df','mexican_df','vietnamese_df','cajun_df','greek_df']
    df_name2 = ['a','b','c','d','e','f','g','h','i','j','k','l','m']
    
    for i, key in enumerate(cat):
    
        string = '''SELECT COUNT(slug), price_level FROM summary WHERE categories LIKE '%{}%' GROUP BY price_level'''.format(key)
        #print(string)
        c.execute(string)
        df_name[i] = pd.DataFrame(data = c.fetchall(), columns = ['count','price_level'])
        d = {'label': [key]*6}
        df_name2[i] = pd.DataFrame(data=d)
        df_name[i] = pd.concat([df_name[i],df_name2[i]], axis = 1)


    nationality_df = pd.concat([df_name[0],df_name[1],df_name[2],df_name[3],df_name[4],df_name[5],df_name[6],
                            df_name[7],df_name[8],df_name[9]], axis = 0)
    nationality_df = pd.concat([nationality_df,df_name[10],df_name[11],df_name[12]], axis = 0)
    return nationality_df

nationality_df = visual_category1()
print(nationality_df.dtypes)
################### Visulization #####################################################

sns.set_style("darkgrid", {'axes.grid': True})
sns.set_context("poster")


g = sns.FacetGrid(nationality_df, margin_titles=True, size=4)
g.map(sns.barplot,'label','count',palette="GnBu_d")
g.add_legend()
g.set_xticklabels(rotation=90)


# Modify the details of the matplotlib Axes g
#g.despine(left=True)
g.set_ylabels('number of restaurants')
g.set_xlabels('types')


In [ ]:
# fields available
# cust_nbr
# div_nbr
# cust_sk
# cust_nm
# addr_ln_1
# addr_ln_2
# city
# st
# zip_cd
# dt_last_ord
# date_ingested
# date_updated
# depth
# limit
# no_menu
# PDF_image
# source
# prospect
# new_refresh
# seed_url
# ingestion_run_id
# current_count
# images
# pages
# ocr



In [ ]:
debug_str(str(soup),"html")

In [ ]:
import itertools
list(itertools.chain.from_iterable(x for x in menuitems))

In [ ]:
 rest = ('"%s' % os.linesep.join([s for s in match.group('rest').splitlines() if s]))
    rest = rest.replace('\n','\r')
    f = csv.writer(open('/mydir/ntuser.csv', 'a'),
                  dialect=csv.excel,
                  delimiter='\t')
    
    

In [ ]:
os.linesep.join([(title,desc,price) for s in menuitems])

In [ ]:
items

In [ ]:
columns =['slug','categories'	,'distance'	,'name'	,'price_level',	'rating',	'review_count',	'url',	'lat',	'lng',
    	'Sp1',	'type' ,	'homeurl',	'resource_id1'	,'resource_id2'	,'lat2',	'lng2']

[print(x +" text,") for x in columns]
